This script extract timing information with RPE based on condition (reward/punishment). Neutral also get extracted but we don't need it and can use the feedback_neutral-nil.txt file 

In [16]:
import pandas as pd
import glob
import os
import csv

In [17]:
os.getcwd()

'/Users/jackiebeltran/Documents/GitHub/RP_Learning/1_scripts/imaging'

In [18]:
reg_list=sorted(glob.glob("../../0_data/regressor/*feedback.txt"))

In [19]:
print("There are " +str(len(reg_list)) + " subjects with regressor data")

There are 83 subjects with regressor data


### obtain timing information

In [20]:
def read_csv_file(file_path):
    
    filename = os.path.splitext(os.path.basename(file_path))[0]
    column_name = filename.split('_')[0]
    
    df = pd.read_csv(file_path, header=None)

    # Rename the column header with subject ID
    df = df.rename(columns={0: column_name})
    
    # create column with subject ID
    df["sub_ID"] = column_name
    
    # rename columns     
    df = df.rename(columns={column_name: "feedback", 1: "sub_ID"})
    
    return df

In [ ]:
# test function
for file in reg_list:
    df = read_csv_file(file)
    print(df)

### obtain RPEs

In [31]:
rpe_list = sorted(glob.glob("/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/RPEs/*csv"))

In [29]:
len(rpe_list)

76

##### load behavioral data to obtain the information on which conditions are which (healthy then mdd)

In [32]:
# clist = sorted(glob.glob('/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/healthy/*csv'))
# clist

clist = sorted(glob.glob('/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/*csv'))
clist

['/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/401.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/402.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/408.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/409.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/410.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/413.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/415.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/416.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/417.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/418.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/419.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/420.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning/0_data/mdd/421.csv',
 '/Users/jackiebeltran/Documents/GitHub/RP_Learning

In [26]:
os.getcwd()

'/Users/jackiebeltran/Documents/GitHub/RP_Learning/1_scripts/imaging'

In [33]:
resDir='../../3_results/model_2/cond_rpe/'

In [34]:
for fb_file in reg_list: # for each feedback timing file
    
    # extract subject ID
    sub_ID = os.path.splitext(os.path.basename(fb_file))[0].split('_')[0]
    
    # outpath to save final txt file
    #fpath = f'/Users/jackiebeltran/Documents/GitHub/RP_Learning/3_results/model_2/feedback_rpe/{sub_ID}_feedback_rpe.txt'
    
    # find matching RPE file from rlist 
    match_rpe = None 
    
    for rpe_file in rpe_list:
        rpe_sub_ID = os.path.splitext(os.path.basename(rpe_file))[0].split('_')[0]
        
        # if i have both the feedback file and rpe file for a subject, make that the matched file
        if sub_ID == rpe_sub_ID: 
            match_rpe = rpe_file 
            break 
            
    if match_rpe:     
        # load both files 
        fb_df = read_csv_file(fb_file)
#         print(fb_df.dtypes)
        rpe_df = pd.read_csv(match_rpe)
        rpe_df = rpe_df[["sub_ID", "RPE"]] # keep columns of interest
               
        df = pd.concat([rpe_df, fb_df], axis=1)
        df = df.loc[:, ~df.columns.duplicated()]                    
        
        # add duration column
        df["duration"] = 4
        
        df["format"] = df['feedback'].astype(str) + "*" + df['RPE'].astype(str) + ":" + df['duration'].astype(str)

#         out_fb = df['format'].tolist()

## here we now want to find the sub_id trial type information output from my behav_data_recoding script        

        df = df[["sub_ID", "format"]]
    
        match_cond = None 
        
        for cond_file in clist:
            cond_sub_ID = os.path.splitext(os.path.basename(cond_file))[0]
            
            if sub_ID == cond_sub_ID:
                match_cond = cond_file
                break
                
        if match_cond:
            format_df = df
            cond_df = pd.read_csv(match_cond)
            
            cond_df = cond_df[["sub_ID", "condition"]]
            
            final_df = pd.concat([cond_df, format_df], axis=1)
            final_df = final_df.loc[:, ~final_df.columns.duplicated()]
            
            # save df with condition & timing
            cond_rpe = pd.DataFrame(final_df)
            cond_rpe.to_csv(f'{resDir}{sub_ID}_condition_rpe.csv',index=False,header=False)  
            
            # creating a list of the different conditions to subset by trial type; 1 = reward, 2=punishment, 3= neutral
            cond=list(final_df.condition.unique()) 
            
            for t in cond:
                df_temp=final_df.loc[(final_df['condition']== t)] 
#                 print(df_temp)
                print("length of dataframe is " + str(len(df_temp))) # each dataframe should have 30 trials
                
                df_temp = df_temp[["format"]]
                
                df_temp.to_csv(f'{resDir}{sub_ID}_{t}_rpe.txt',index=False,header=False)
                print(f'condition_rpe for {sub_ID} stored at {resDir}')

length of dataframe is 30
condition_rpe for 401 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 401 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 401 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 402 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 402 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 402 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 408 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 408 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 408 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 409 stored at ../../3_results/model_2/cond_rpe/
length of dataframe is 30
condition_rpe for 409 stored at ../../3_results/model_